In [36]:
#IMPORTAÇÃO
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain



In [38]:
llm = ChatOpenAI(model="gpt-3.5-turbo")



In [11]:
#loader 
loader = WebBaseLoader("https://defensoria.mg.def.br/defensoria-publica-reune-especialistas-para-discutir-o-papel-do-conselho-tutelar-na-protecao-da-crianca-e-do-adolescente-vitima-de-violencia/")

docs = loader.load()
len(str(docs)), docs

(11020,
 [Document(metadata={'source': 'https://defensoria.mg.def.br/defensoria-publica-reune-especialistas-para-discutir-o-papel-do-conselho-tutelar-na-protecao-da-crianca-e-do-adolescente-vitima-de-violencia/', 'title': '  DPMG reúne especialistas para discutir o papel do Conselho Tutelar na proteção da criança e do adolescente vítima de violência', 'language': 'pt-br'}, page_content='\n\n\n\n\n\n\n  DPMG reúne especialistas para discutir o papel do Conselho Tutelar na proteção da criança e do adolescente vítima de violência\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\nVisão\nMissão\nValores\n\n\n\n\n\n\n\n\n\nFale com a Defensoria\n\n\n\n\n\t\t\t\t\t\t\t\tZoom\u2002-A\u2002\n\t\t\t\t\t\t\t\t+A\u2002|\u2002\n\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\tContraste\u2002|\u2002\n\t\t\t\t\t\t\t\n\nMapa do Site\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDPMG\n\nPerfil\nPlanejamento Estratégico\nEstrutura Org

In [15]:
#text_splitter 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


text_splitter, splits, len(splits)

(<langchain_text_splitters.character.RecursiveCharacterTextSplitter at 0x7b7908aa7070>,
 [Document(metadata={'source': 'https://defensoria.mg.def.br/defensoria-publica-reune-especialistas-para-discutir-o-papel-do-conselho-tutelar-na-protecao-da-crianca-e-do-adolescente-vitima-de-violencia/', 'title': '  DPMG reúne especialistas para discutir o papel do Conselho Tutelar na proteção da criança e do adolescente vítima de violência', 'language': 'pt-br'}, page_content='DPMG reúne especialistas para discutir o papel do Conselho Tutelar na proteção da criança e do adolescente vítima de violência\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\nVisão\nMissão\nValores\n\n\n\n\n\n\n\n\n\nFale com a Defensoria\n\n\n\n\n\t\t\t\t\t\t\t\tZoom\u2002-A\u2002\n\t\t\t\t\t\t\t\t+A\u2002|\u2002\n\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\tContraste\u2002|\u2002\n\t\t\t\t\t\t\t\n\nMapa do Site\n\n\n\n\n\n\n\n\n\n\n\n\

In [22]:
#vectorstore
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()



(<langchain_chroma.vectorstores.Chroma at 0x7b78d32460b0>,
 VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7b78d32460b0>))

In [44]:
#prompt
system_prompt = (
	"Você é um assistente para tarefas de resposta a perguntas."
 	"Use os seguintes trechos de contexto recuperado para responder"
  	"à pergunta. Se você não sabe a resposta, diga que não sabe." 
   	"Use no máximo três frases e mantenha a resposta concisa."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [43]:
rag_chain.invoke({"input": "Sobre o que é o evento?"})["answer"]

'O evento abordará os "Aspectos práticos da revelação espontânea" e está relacionado à Lei nº 13.431/17, também conhecida como Lei da Escuta Protegida, que estabelece o Sistema de Garantia de Direitos da Criança e do Adolescente vítima ou testemunha de violência.'

In [45]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///memory.db")

In [46]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory

In [47]:
runnable_with_history = RunnableWithMessageHistory(
    llm,
    get_session_history,
)

In [49]:
runnable_with_history.invoke(
    [HumanMessage(content="Estou estudando sobre chat")],
    config={"configurable": {"session_id": "1"}},
)

AIMessage(content='Que legal! Os chatbots estão se tornando cada vez mais populares e são uma ferramenta muito útil para empresas e usuários. Se tiver alguma dúvida ou quiser saber mais sobre o assunto, estou aqui para ajudar.', response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 40, 'total_tokens': 95}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-de790351-b156-47f2-bb92-c5d63218bb05-0', usage_metadata={'input_tokens': 40, 'output_tokens': 55, 'total_tokens': 95})

In [50]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who speaks in {language}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm

runnable_with_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [51]:
runnable_with_history.invoke(
    {"language": "portugues", "input": "Ola eu sou o lucas"},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='Olá Lucas! Como posso ajudar você hoje?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 36, 'total_tokens': 48}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7dced309-fbe3-469e-bd50-9251d1b6ec25-0', usage_metadata={'input_tokens': 36, 'output_tokens': 12, 'total_tokens': 48})

In [52]:
runnable_with_history.invoke(
    {"language": "portugues", "input": "Qual o meu nome?"},
    config={"configurable": {"session_id": "2"}},
)

AIMessage(content='Seu nome é Lucas, como você mesmo disse anteriormente. Posso ajudar com mais alguma coisa?', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 61, 'total_tokens': 84}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0c8acabf-64f5-42ad-a2b3-f9f0fb39b500-0', usage_metadata={'input_tokens': 61, 'output_tokens': 23, 'total_tokens': 84})